# 04 - Test du serving MLflow

Ce notebook valide le modèle servi via Docker/MLflow.

## Prérequis

1. Construire l'image : `docker build -t credit-scoring-model .`
2. Lancer le conteneur : `docker run -p 1234:1234 credit-scoring-model`
3. Vérifier que le serveur répond : `curl http://localhost:1234/health`

## Notes importantes

- Le dataset de test est déjà préparé (même format que lors de l'entraînement)
- Seule conversion nécessaire : les colonnes lues comme `object` par pandas doivent être converties en numériques
- Les NaN sont conservés (LightGBM les supporte nativement)
- Aucun autre retraitement n'est nécessaire



In [13]:
import sys
import os

# 1. On ajoute le dossier racine du projet au "path" de Python
# Cela permet d'importer tout ce qui se trouve dans le dossier 'src'
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# 2. On importe la fonction de nettoyage
from src.model_utils import clean_feature_names

print("Fonction 'clean_feature_names' importée avec succès !")

✅ Fonction 'clean_feature_names' importée avec succès !


c:\Users\mathi\OneDrive\Bureau\ESAIP\ING4\MAJEUR\Majeur_IA310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
import pandas as pd
import requests
import json
import numpy as np

# Configuration
URL = "http://localhost:1234/invocations"
TEST_PATH = "../datasets/final/train_enriched.csv"

# 1. Charger l'échantillon brut
print("Chargement des données...")
sample = pd.read_csv(TEST_PATH).head(5)

# 2. APPLICATION DU NETTOYAGE (C'est l'étape qui manquait !)
print("Nettoyage des noms de colonnes (Transformation en format LightGBM)...")
sample_clean = clean_feature_names(sample)

# 3. Retirer les colonnes inutiles (Target/ID)
# Note: On utilise les noms NETTOYÉS maintenant
cols_to_drop = ['TARGET', 'SK_ID_CURR', 'index', 'Unnamed_0']
existing_cols = [c for c in cols_to_drop if c in sample_clean.columns]
sample_clean = sample_clean.drop(columns=existing_cols)

# 4. Gestion des NaN pour le JSON (Via l'astuce to_json)
# On laisse Pandas gérer la conversion NaN -> null
json_str = sample_clean.to_json(orient="split")
split_dict = json.loads(json_str)

# 5. Envoi
payload = {"dataframe_split": split_dict}

print(f"📡 Envoi de {len(sample_clean)} clients au modèle...")
print(f"   Exemple de colonne nettoyée : {sample_clean.columns[0]}") 
# Vous devriez voir des underscores (_) au lieu des espaces

try:
    response = requests.post(URL, json=payload)
    
    if response.status_code == 200:
        print("\nSUCCÈS")
        print(response.json())
    else:
        print(f"\nErreur API ({response.status_code}) :")
        print(response.text)

except Exception as e:
    print(f"\nErreur : {e}")

Chargement des données...
Nettoyage des noms de colonnes (Transformation en format LightGBM)...
📡 Envoi de 5 clients au modèle...
   Exemple de colonne nettoyée : NAME_CONTRACT_TYPE

✅ SUCCÈS TOTAL ! Le modèle a répondu :
{'predictions': [1, 0, 0, 0, 0]}


In [19]:
test = pd.read_csv('../datasets/final/train_labels.csv').head(5)


In [20]:
test.head(5)

,TARGET
0,1
1,0
2,0
3,0
4,0
